In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img1 = cv2.imread('../data/GEarth/left.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('../data/GEarth/front_left.png', cv2.IMREAD_GRAYSCALE)
img3 = cv2.imread('../data/GEarth/front.png', cv2.IMREAD_GRAYSCALE)

images = [img1, img2, img3]


In [6]:
def stitch_images(images):
    # Check and convert images
    processed_images = []
    for img in images:
        if img is None:
            print("One of the input images is None.")
            return None
        
        # Ensure the image is in BGR format
        if len(img.shape) == 2:  # Image is grayscale
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        elif img.shape[2] == 4:  # Image is RGBA
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
        
        processed_images.append(img)

    # Create a Stitcher instance
    stitcher = cv2.Stitcher.create(cv2.Stitcher_PANORAMA)
    
    # Perform the stitching process
    status, stitched_image = stitcher.stitch(processed_images)
    
    if status == cv2.Stitcher_OK:
        print("Images stitched successfully.")
        # Display the stitched image
        return stitched_image
    
res_stitch = stitch_images(images)
cv2.imwrite('../results/pano.png', res_stitch)

Images stitched successfully.


True